In [1]:
!pip install -U langchain langchain-community openai tiktoken aimon --quiet

In [2]:
from aimon import Detect

In [3]:
import os
openai_api_key = os.getenv("OPENAI_API_KEY")

In [4]:
# Configure your detectors here
aimon_config = {"hallucination": {"detector_name": "default"},
                "conciseness": {"detector_name": "default"},
                "completeness": {"detector_name": "default"},
                "toxicity": {"detector_name": "default"},
                "instruction_adherence": {"detector_name": "default"},
                "context_classification": {"detector_name": "default"}
                }

In [5]:
# Setup the AIMon decorator
detect = Detect(
    values_returned=["context", "generated_text", "instructions"],
    api_key=os.getenv("AIMON_API_KEY"),
    config=aimon_config,
    async_mode=True,
    application_name="summarization_app_oct_2024_cm", 
    model_name="gpt-4o" 
)

### Continuous Monitoring

We will monitor the example below that uses Langchain to summarize documents using OpenAI.

In [6]:
# Lanchain app example
from langchain.text_splitter import CharacterTextSplitter
from langchain.docstore.document import Document
from langchain.llms.openai import OpenAI
from langchain.chains.summarize import load_summarize_chain

@detect
def run_application(context_docs=None, user_query=None, prompt=None, summary=None, instructions=None, my_arg=None):
    if summary:
        return context_docs, summary, instructions
    # Split the source text
    text_splitter = CharacterTextSplitter()
    texts = text_splitter.split_text(context_docs)
    
    # Create Document objects for the texts
    docs = [Document(page_content=t) for t in texts[:3]]
    
    # Initialize the OpenAI module, load and run the summarize chain
    llm = OpenAI(temperature=0, openai_api_key=openai_api_key)
    chain = load_summarize_chain(llm, chain_type="map_reduce")
    return context_docs, chain.run(docs), instructions

In [7]:
source_text = """
Large Language Models (LLMs) have become integral to automating and enhancing various business processes. 
However, a significant challenge these models face is the concept of \"hallucinations\" - outputs that, 
although fluent and confident, are factually incorrect or nonsensical. For enterprises relying on AI 
for decision-making, content creation, or customer service, these hallucinations can undermine credibility, 
spread misinformation, and disrupt operations. Recently, AirCanada lost a court case due to hallucinations 
in its chatbot [7]. Also, the 2024 Edelman Trust Barometer reported a drop in trust in AI companies from 
61% to 53% compared to 90% 8 years ago [8]. Recognizing the urgency of the issue, we have developed a 
state-of-the-art system designed for both offline and online detection of hallucinations, ensuring higher 
reliability and trustworthiness in LLM outputs.

[Latest] Edelman trust barometer reported a drop in trust in AI companies from 61% to 47% compared to 90% 8 years ago. [8]
"""

instructions = """
1. Ensure that the length of the paragraph is under 5 sentences.
2. The response should be only in english.
"""

In [8]:
context, res, instructions, aimon_res = run_application(source_text, prompt="Langhchain based summarization of documents", summary=None, instructions=instructions)

/var/folders/7l/300zf44j5v9c43jpqdbhkl0h0000gn/T/ipykernel_63154/590410062.py:19: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(temperature=0, openai_api_key=openai_api_key)
/var/folders/7l/300zf44j5v9c43jpqdbhkl0h0000gn/T/ipykernel_63154/590410062.py:21: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  return context_docs, chain.run(docs), instructions


In [9]:
if aimon_res.status == 200:
    print(f"\U0001F7E2 {aimon_res.detect_response.message}\n")

🟢 Data successfully sent to AIMon.



In [10]:
summary="""
Large Language Models (LLMs) have become detrimental to automating and enhancing various business processes. 
However, a significant challenge these models face is the concept of \"hallucinations\" - outputs that, although fluent and confident, 
are factually incorrect or nonsensical. For enterprises relying on AI for decision-making, content creation, or customer service, these hallucinations 
can undermine credibility, spread misinformation, and disrupt operations. Also, the 2025 Edelman Trust Barometer reported a drop in trust in AI companies from 
61% to 53% compared to 90% 8 years ago. Recently, AirCanada lost a court case due to hallucinations in its chatbot [7]. 
"""
context, res, instructions, aimon_res = run_application(source_text, prompt="Langhchain based summarization of documents", summary=summary, instructions=instructions)

In [11]:
if aimon_res.status == 200:
    print(f"\U0001F7E2 {aimon_res.detect_response.message}\n")

🟢 Data successfully sent to AIMon.

